<a href="https://colab.research.google.com/github/deekshakoul/TSP-via-GA/blob/main/TSP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import math
import numpy as np
import matplotlib.pyplot as plt
import random as rnd

READ FILE - contains coordinates of 100 cities

In [ ]:
f = open('/content/City_Coordinates.txt','r') #PATH OF FILE City_Coordinates.txt
text = f.readlines()
f.close()

X = []
Y = []
for i in range(len(text)):
  x,y = text[i].split(',')
  X.append(int(x))
  Y.append(int(y.strip()))
city_map = dict(enumerate(zip(X,Y))) # A dict mapping of city index to location
cities = list(city_map.keys()) # city - index

Initialize population

In [ ]:
def initialize_pop(pop_size=10):
    population = []
    for i in range(pop_size):
        tour = cities.copy()
        rnd.shuffle(tour) 
        population.append(tour)
    return population

Fitness - total round trip distance covered from start of city to back

In [ ]:
def distance(c1,c2):
  return math.sqrt( (c1[0]-c2[0])**2 + (c1[1]-c2[1])**2 )  

In [ ]:
def fitness(solution):
    total_distance = 0
    
    for i in range(len(solution)-1):
        p1 = city_map[solution[i]]
        p2 = city_map[solution[i+1]]
        total_distance += distance(p1, p2)
    p1 = city_map[solution[-1]]
    p2 = city_map[solution[0]]
    total_distance += distance(p1, p2)
       
    return total_distance

In [ ]:
def selection(population):
  fit_values = list(map(fitness, population))
  inv_fitness = [max(fit_values) - x for x in fit_values]
  weights = [x / sum(inv_fitness) for x in inv_fitness]
  select_p = rnd.choices(population, weights)[0] #most fittest tour is selected
  return select_p

Implementation of cross-overs

In [ ]:
def cx(p1, p2): #take first element from p1
  child = [-1]*len(p1)
  first_index = 0 #index to start first cycle
  val = p1[first_index]
  child[first_index] = val
  flag=True
  turn = 1
  t1 = p1
  t2 = p2

  while (flag):
    #ONE CYCLE
    while (t2.index(val)!= first_index):
      ind = t2.index(val) #at this index place value of p1/t1
      val = t1[ind]
      child[ind] = val
    # print(child,"after inner whiile")  
    if -1 not in child:
      flag = False
    else:
      first_index = child.index(-1)
      if turn == 1: #we have started cycle with p1 ...now we need to chnage this
        t1 = p2
        t2 = p1
        turn = 2 # now start with other parent
      else:
        t1 = p1
        t2 = p2
        turn = 1       
      val = t1[first_index]
      child[first_index] = val
  return child

In [ ]:
p1 = [1,2,3,4,5]
p2 = [4,3,1,5,2]
#child should be 1 2 3 4 5
cx(p1,p2)

In [ ]:
def pmx(p1, p2): #changes cutout parts of p1 and replace with that of p2, rest of genes of p1 ordered accordingly
  c1 = rnd.randint(0, len(p1)-2)
  c2 = rnd.randint(c1+1, len(p2)-1)
  # print(c1,c2)
  child = [-1]*len(p1)
  child[c1:c2+1] = p2[c1:c2+1]
  for i in range(0,c1):
    if p1[i] not in child:
      child[i] = p1[i]
    else:
      ind = child.index(p1[i]) #6  
      pp = p1[ind] #value at index: ind i.e 6
      while (pp in child):
        ind = child.index(pp) #5
        pp = p1[ind] #value at index: ind i.e 5
      child[i] = pp
  
  for i in range(c2+1,len(p1)):
    if p1[i] not in child:
      child[i] = p1[i]
    else:
      ind = child.index(p1[i]) #6  
      pp = p1[ind] #value at index: ind i.e 6
      while (pp in child):
        ind = child.index(pp) #5
        pp = p1[ind] #value at index: ind i.e 5
      child[i] = pp
  
  return child

In [ ]:
def ox1(p1, p2): #copy cut points data of P1
  c1 =rnd.randint(0, len(p1)-2)
  c2 =rnd.randint(c1+1, len(p2)-1) #include c2 too
  # print(c1," # ",c2)
  rem = [i for i in range(c2+1,len(p1))] 
  rem.extend([i for i in range(c1)]) 
  n = len(rem) #indices that i need to copy in child
  rem.extend([i for i in range(c1,c2+1)])

  child = [-1]*len(p1)
  child[c1:c2+1] = p1[c1:c2+1]

  for i in range(n):
    j=i
    while p2[rem[j]] in child:
      j = j + 1  
    child[rem[i]] = p2[rem[j]]

  return child

In [ ]:
def ox2(p1,p2): 
  k = rnd.randint(4,40)#len(p1) #random number of positions to conisder
  n = sorted(rnd.sample(range(0,len(p1)), k))
  #n=[2,4,6] #need to have indieces in sorted order.....imp
  #print(n)
  cities = [p2[i] for i in n] #maintain order of p2
  #print("city", cities)
  cities_index_p1 = [p1.index(i) for i in cities] 
  cities_index_p1 = sorted(cities_index_p1)
  child = p1
  i=0
  for c in cities_index_p1:
    child[c] = cities[i]
    i=i+1
    
  #print(child)
  return child

In [ ]:
def pos(p1,p2):
  k= rnd.randint(3,20)#len(p1)
  n = rnd.sample(range(0,len(p1)), k) #generates position
  #print(n)
  child = p1
  for i in n:
    t2 = p2[i]
    t1 = p1[i]
    ind = p1.index(t2)
    p1[i] = t2
    p1[ind] = t1
  return child  

In [ ]:
def mpx(p1,p2):
  k = rnd.randint(3,20) #substring length
  c1 = rnd.randint(0, len(p2)-k) 
  part_p1 = p1[c1:c1+k]
  # child = p2
  child = [i for i in p2 if i not in part_p1]
  for i in range(k):
    child.insert(0,part_p1[k-i-1])
  # print(child)
  return child

In [ ]:
def apc(p1,p2):
  #start from parent 1
  child = []
  for i in range(len(p1)):
    if p1[i] not in child:
      child.append(p1[i])
    if p2[i] not in child:
      child.append(p2[i])  
  return child    

In [ ]:
def mutation(parent):
    posA = rnd.randint(0, len(parent)-1)
    posB = rnd.randint(0, len(parent)-1)
    
    child = parent.copy()
    child[posA] = parent[posB]
    child[posB] = parent[posA]
    
    return child

In [ ]:
def GA(pop_size, max_generations, pcross, pmut, elite_size):
    # Initialize the pops
    pops = initialize_pop(pop_size)
    best_result = {}

    for i in range(max_generations):
        print("Itertaion: ",i+1)
        fitness_pop = list(map(fitness, pops))

        opt_fitness = min(fitness_pop)
        opt_fit_index = fitness_pop.index(opt_fitness)
        opt_individual = pops[opt_fit_index] #the tour that has the min distance

        if i == 0:
            best_result['fitness'] = opt_fitness
            best_result['tour'] = opt_individual
        else:
            if opt_fitness <= best_result['fitness']:
                best_result['fitness'] = opt_fitness
                best_result['tour'] = opt_individual
                print('Best Tour length at Generation: ', i+1, " with best fitness as : ", opt_fitness)

        new_pops = []
        for i in range(elite_size):
            new_pops.append(selection(pops))
        
        for i in range(pop_size-elite_size):
            parent1 = selection(pops)
            parent2 = selection(pops)

            # crossover with some probability
            if rnd.random() <= pcross:
                child = ox1(parent1, parent2)
            else: # copy one of the parents if crossover doesnt happend
                child = rnd.choice([parent1, parent2])

            if rnd.random() <= pmut:
                child = mutation(child)

            # Add offspring to new population
            new_pops.append(child)

        pops = new_pops
    return best_result

In [ ]:
pop_size = 50
max_generations = 500
crossover_prob = 0.95
mutate_prob = 0.01
elite=20

In [ ]:
result = GA(pop_size, max_generations, crossover_prob, mutate_prob,elite)
result

In [ ]:
result